# DAT-500 Project
This notebook is the final project
Project description: Create a machine learning algorithm capable of training a big dtaset of parquet files containing cryptocurrency history from Binance, save the models for later usage and evaluation.

### Fewer settings to be considered:
On remote server check HADOOP_HOME environment variable is properly configured
default hadoop host is hdfs://master:9000
core-site.xml and look for xml element fs.defaultFS
#### List of hadoop remote data files directories:

<font color="Green">Files</font>

- /Data/Abcnews-date-text.csv
- /Data/Binance.csv
- /Data/CoinMarketCap.csv
- /Data/QuandlData.csv
- /Data/cointelegraph_news1-1000_content.csv
- /Data/cointelegraph_news1-1000_head.csv

<font color="Green">Directories </font>

- /Data/Binance_Parquet
- /Data/Binance_Parquet/ADA-BNB.parquet
- /Data/Binance_Parquet/ADA-BTC.parquet
- /Data/Binance_Parquet/ADA-BUSD.parquet


etc/

In [193]:
import os
os.environ["HADOOP_HOME"]

'/usr/local/hadoop'

In [194]:
!cat /usr/local/hadoop/etc/hadoop/core-site.xml

<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
  <property>
    <name>fs.default.name</name>
    <value>hdfs://master:9000</value>
  </property>
  <property>
    <name>hadoop.proxyuser.ubuntu.hosts</name>
    <value>*</value>
  </property>
  <property>
    <name>hadoop.proxyuser.ubuntu.gro

In [195]:
!pip3.8 install pyarrow
!pip3.8 install hdfs
!pip3.8 install tqdm
!pip3.8 install sklearn
!pip3.8 install matplotlib
!pip3.8 install sparkflow

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [196]:
!ls -ltr /home/ubuntu/

!ls /home/ubuntu/.hdfscli.cfg

!hadoop fs -ls /Data
!hadoop fs -ls /Data/Binance_Parquet

total 60
drwxrwxr-x 3 ubuntu ubuntu  4096 Mar 27 16:07 pkg
drwxr-xr-x 2 ubuntu ubuntu  4096 Apr 10 19:03 dis_materials
-rw-rw-r-- 1 ubuntu ubuntu   225 Apr 11 20:13 spark-yarn.py
drwxrwxr-x 9 ubuntu ubuntu  4096 Apr 12 10:46 python-spark-tutorial
drwxrwxr-x 3 ubuntu ubuntu 36864 Apr 12 11:14 Project_Dataset
-rwxrwxr-x 1 ubuntu ubuntu   143 Apr 19 06:37 jn.sh
drwxrwxr-x 4 ubuntu ubuntu  4096 Apr 19 10:11 notebooks
/home/ubuntu/.hdfscli.cfg
Found 8 items
-rw-r--r--   2 ubuntu supergroup   60220782 2020-04-11 19:51 /Data/Abcnews-date-text.csv
-rw-r--r--   2 ubuntu supergroup  881532985 2020-04-11 19:52 /Data/Binance.csv
drwxr-xr-x   - ubuntu supergroup          0 2020-04-12 11:36 /Data/Binance_Parquet
-rw-r--r--   2 ubuntu supergroup     894659 2020-04-11 19:52 /Data/CoinMarketCap.csv
-rw-r--r--   2 ubuntu supergroup     619360 2020-04-11 19:52 /Data/QuandlData.csv
-rw-r--r--   2 ubuntu supergroup   87552220 2020-04-11 19:51 /Data/cointelegraph_news1-1000_content.csv
-rw-r--r--   2 ubuntu

-rw-r--r--   2 ubuntu supergroup    4702174 2020-04-12 11:21 /Data/Binance_Parquet/BNB-USDS.parquet
-rw-r--r--   2 ubuntu supergroup   44925392 2020-04-12 11:21 /Data/Binance_Parquet/BNB-USDT.parquet
-rw-r--r--   2 ubuntu supergroup     388907 2020-04-12 11:21 /Data/Binance_Parquet/BNBBEAR-BUSD.parquet
-rw-r--r--   2 ubuntu supergroup     561924 2020-04-12 11:21 /Data/Binance_Parquet/BNBBEAR-USDT.parquet
-rw-r--r--   2 ubuntu supergroup     340652 2020-04-12 11:21 /Data/Binance_Parquet/BNBBULL-BUSD.parquet
-rw-r--r--   2 ubuntu supergroup     554120 2020-04-12 11:21 /Data/Binance_Parquet/BNBBULL-USDT.parquet
-rw-r--r--   2 ubuntu supergroup   20913047 2020-04-12 11:21 /Data/Binance_Parquet/BNT-BTC.parquet
-rw-r--r--   2 ubuntu supergroup     580459 2020-04-12 11:21 /Data/Binance_Parquet/BNT-BUSD.parquet
-rw-r--r--   2 ubuntu supergroup   17227249 2020-04-12 11:21 /Data/Binance_Parquet/BNT-ETH.parquet
-rw-r--r--   2 ubuntu supergroup     858646 2020-04-12 11:21 /Data/Binance_Parquet/BNT

-rw-r--r--   2 ubuntu supergroup   14477314 2020-04-12 11:28 /Data/Binance_Parquet/NPXS-BTC.parquet
-rw-r--r--   2 ubuntu supergroup   18122530 2020-04-12 11:28 /Data/Binance_Parquet/NPXS-ETH.parquet
-rw-r--r--   2 ubuntu supergroup    1254875 2020-04-12 11:28 /Data/Binance_Parquet/NPXS-USDC.parquet
-rw-r--r--   2 ubuntu supergroup    5983328 2020-04-12 11:28 /Data/Binance_Parquet/NPXS-USDT.parquet
-rw-r--r--   2 ubuntu supergroup   14458236 2020-04-12 11:28 /Data/Binance_Parquet/NULS-BNB.parquet
-rw-r--r--   2 ubuntu supergroup   30486124 2020-04-12 11:28 /Data/Binance_Parquet/NULS-BTC.parquet
-rw-r--r--   2 ubuntu supergroup   22702113 2020-04-12 11:28 /Data/Binance_Parquet/NULS-ETH.parquet
-rw-r--r--   2 ubuntu supergroup   21005033 2020-04-12 11:28 /Data/Binance_Parquet/NULS-USDT.parquet
-rw-r--r--   2 ubuntu supergroup    7568762 2020-04-12 11:28 /Data/Binance_Parquet/NXS-BNB.parquet
-rw-r--r--   2 ubuntu supergroup   17716595 2020-04-12 11:28 /Data/Binance_Parquet/NXS-BTC.parquet

In [197]:
import os
import tqdm
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkFiles
import pyarrow as pa
import pyarrow.parquet as pq

Configure verbosity level for Tensorflow fitting models:


Level | Level for Humans | Level Description                  
 -------|------------------|------------------------------------ 
  0     | DEBUG            | [Default] Print all messages       
  1     | INFO             | Filter out INFO messages           
  2     | WARNING          | Filter out INFO & WARNING messages 
  3     | ERROR            | Filter out all messages 

In [184]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}

In [222]:
#set the hadoop hostname to read files and dir
hdp_master = 'hdfs://master:9000'
parquet_dir = '/Data/Binance_Parquet'
#.master('yarn') \
session = SparkSession.builder \
    .appName('DAT500-CRYPTO') \
    .getOrCreate()


In [213]:
session.conf.get("spark.sql.parquet.compression.codec")
#session.conf.set("spark.sql.parquet.compression.codec","uncompressed")

'snappy'

In [223]:
#session.stop()
#sc = pyspark.SparkContext()
session.sparkContext

<SparkContext master=yarn appName=DAT500-CRYPTO>

In [79]:
from hdfs import Config
client = Config().get_client('dev')
parquet_files = client.list(parquet_dir)
parquet_files = [x for x in parquet_files if '.parquet' in x]

In [88]:
parquet_files[0:5]

['ADA-BNB.parquet',
 'ADA-BTC.parquet',
 'ADA-BUSD.parquet',
 'ADA-ETH.parquet',
 'ADA-PAX.parquet']

In [81]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [179]:
%reload_ext autoreload

In [198]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

import crypto_utils as cu
from crypto_params import *

In [199]:
def try_create_dir(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def try_create_dirs(paths):
    for path in paths:
        try_create_dir(path)

def create_tensorboard(model_name:str):
    checkpoint = ModelCheckpoint(os.path.join("results", model_name), save_weights_only=True, save_best_only=True, verbose=1)
    tensorboard = TensorBoard(log_dir=os.path.join("logs", model_name))
    return checkpoint, tensorboard


try_create_dirs( (RESULTS_PATH, LOG_PATH, DATA_PATH) )

In [ ]:
EPOCHS = 2 #300    

models = []
pfiles = parquet_files[0:20]
for parquet_file in tqdm.tqdm(pfiles):
    currency = os.path.splitext(parquet_file)[0]
    df = pq.read_table(f"{hdp_master}/Data/Binance_Parquet/{parquet_file}")
    df = df.to_pandas()
    data = cu.load_data(currency,df,'hour', N_STEPS, window_offset=WINDOW_OFFSET, test_size=TEST_SIZE, feature_cols=FEATURE_COLS,log =False)
    
    model = cu.create_model(N_STEPS, loss=LOSS, units=N_UNITS, cell=CELL, n_layers=N_LAYERS,dropout=DROPOUT, optimizer=OPTIMIZER)
    checkpoint, tensorboard = create_tensorboard(currency)
    trained_model = model.fit(data["X_train"], data["y_train"],
                        batch_size=BATCH_SIZE,
                        epochs=EPOCHS,
                        validation_data=(data["X_test"], data["y_test"]),
                        callbacks=[checkpoint, tensorboard],
                        verbose=3)
    
    #json_model = trained_model.model.to_json()
    #cu.save_model(trained_model.model,data, RESULTS_PATH,currency)
    #df_model = spark.read.json(session.parallelize([json_model]))
    
    model_dict = {'currency':currency,'data':data,'model':trained_model.model}
    models.append(model_dict)

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1/2

Epoch 00001: val_loss improved from inf to 0.00518, saving model to results/ADA-BNB
Epoch 2/2


  5%|▌         | 1/20 [00:44<13:57, 44.08s/it]


Epoch 00002: val_loss improved from 0.00518 to 0.00128, saving model to results/ADA-BNB
Epoch 1/2

Epoch 00001: val_loss improved from inf to 0.00564, saving model to results/ADA-BTC
Epoch 2/2


 10%|█         | 2/20 [01:39<14:12, 47.37s/it]

...

 75%|███████▌  | 15/20 [11:35<04:19, 51.98s/it]


Epoch 00002: val_loss improved from 0.00354 to 0.00257, saving model to results/AGI-BNB
Epoch 1/2

Epoch 00001: val_loss improved from inf to 0.00317, saving model to results/AGI-BTC
Epoch 2/2


 80%|████████  | 16/20 [12:25<03:24, 51.21s/it]


Epoch 00002: val_loss did not improve from 0.00317
Epoch 1/2

Epoch 00001: val_loss improved from inf to 0.01892, saving model to results/AGI-ETH
Epoch 2/2

Epoch 00002: val_loss improved from 0.01892 to 0.00446, saving model to results/AGI-ETH


 85%|████████▌ | 17/20 [13:23<02:39, 53.19s/it]

## Plot results

In [ ]:
cu.plot_graphs(models,100,WINDOW_OFFSET)

## Read a Model from Json (Spark to ML Model)

In [107]:
from tensorflow import keras
mdl = keras.models.model_from_json(a)

In [ ]:
#cu.plot_graph(trained_model.model,data,currency,100)


#data = [('First', 1), ('Second', 2), ('Third', 3), ('Fourth', 4), ('Fifth', 5)]
#df = session.createDataFrame(data)
#df.write.csv(f"{hdp_master}/example.csv")


#session.read.json(session.sparkContext.parallelize([json_model]))
#df_model = pyspark.read.json(session.parallelize([json_model]))




#a = session.read.csv(f"{hdp_master}/Abcnews-date-text.csv")

#df_load = session.read.csv(f"{hdp_master}/example.csv")
#df_load.show()